In [ ]:
"""
Purpose: To check the progress of version 4
pods running

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-06-01 20:41:19,552 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 20:41:19,555 - settings - Setting database.user to celiib
INFO - 2021-06-01 20:41:19,556 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 20:41:19,560 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-06-01 20:41:19,561 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 20:41:19,576 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-06-01 20:41:20,092 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 20:41:20,097 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 20:41:20,098 - settings - Setting database.user to celiib
INFO - 2021-06-01 20:41:20,099 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 20:41:20,099 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 20:41:20,100 - settings - Setting database.user to celiib
INFO - 2021-06-01 20:41:20,101 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 20:41:20,103 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 20:41:20,616 - settings - Setting enable_python_native_blobs to True


In [3]:
import datajoint as dj
version = 88
nucleus_table_ids = du.nucleus_info_table(version = 88) & "segment_id>0"
minnie, schema = du.configure_minnie_vm()


INFO - 2021-06-01 20:41:20,788 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 20:41:20,790 - settings - Setting database.user to celiib
INFO - 2021-06-01 20:41:20,791 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 20:41:20,794 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 20:41:21,366 - settings - Setting enable_python_native_blobs to True


# The list of segments

In [4]:
minnie.NucleiSegmentsRun4()

segment_id segment id for those to be decimated
864691131559524855
864691131658969700
864691131705161142
864691131771477291
864691131800703080
864691131820044899
864691131828706323
864691131834278152
864691131848667157
864691131906133483


# Decimation

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decimation'")
curr_table

# Soma Extraction

In [ ]:
minnie.BaylorSegmentCentroid()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__baylor_segment_centroid'")
curr_table

In [ ]:
len(minnie.BaylorSegmentCentroid()) - 102595

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
key_source =  (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (
                                                                  du.proofreading_segment_id_restriction()
                                                                 ))
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.BaylorSegmentCentroid())

# 4) Decomposition

In [ ]:
key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")
                  & du.proofreading_segment_id_restriction())
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.Decomposition())

In [ ]:
len(minnie.Decomposition()) - 77723

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
curr_table
#du.restrict_jobs_table_by_error_substring(curr_table,"SystemExit: SIGTERM received").delete()

# DecompositionAxon

In [ ]:
"""
NOTE: quit some jobs at 11:20 so could erase those from the jobs 
schema

"""

In [ ]:
"""
Reminder have to eliminate the jobs that were ended prior to running
the new batch

Already did using: 
# curr_table = (minnie.schema.jobs & "table_name='__decomposition_axon'")
# (curr_table & "timestamp<'2021-05-17 14:00:03'").delete()

"""

In [ ]:
len(minnie.DecompositionAxon())

In [ ]:
len(minnie.Decomposition()) - 75967

In [ ]:
minnie.Decomposition()

In [ ]:
key_source = minnie.Decomposition() & du.proofreading_segment_id_restriction()
(dj.U("segment_id") & key_source) - (dj.U("segment_id") & minnie.DecompositionAxon())

# Proofreading Table

In [ ]:
(minnie.AutoProofreadNeurons4() & "axon_length > 1000" & "cell_type_predicted='excitatory'" )

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__auto_proofread_neurons4'")
curr_table#.delete()

# Inspecting Certain Neurons

In [ ]:
(minnie.AutoProofreadNeurons4() & "spine_category='densely_spined'") & "axon_length=0"

In [ ]:
((minnie.DecompositionAxon() & "axon_length>1000") & 
(dj.U("segment_id","split_index") & (minnie.AutoProofreadNeurons4() & "spine_category='densely_spined'" & "axon_length=0")).proj())

In [ ]:
minnie.AutoProofreadStats4() & dict(segment_id=864691135495637444)

In [ ]:
minnie.AutoProofreadNeurons4() & dict(segment_id=864691135495637444)

In [ ]:
du.plot_proofread_neuron(864691134884777978,
                        split_index=0,
                        plot_proofreading_synapses=True)

In [ ]:
dir(du)

In [ ]:
decomp_path = (minnie.DecompositionAxon() & dict(segment_id=864691135495637444)).fetch1("decomposition")
neuron_obj = du.filepath_to_neuron_obj(decomp_path)

In [ ]:
import neuron_visualizations as nviz
nviz.plot_axon(neuron_obj)

In [ ]:
import proofreading_utils as pru
pru.proofread_neuron_class_predetermined(neuron_obj,
    inh_exc_class="excitatory",
    perform_axon_classification=False,
    plot_limb_branch_filter_with_disconnect_effect=True,
    high_fidelity_axon_on_excitatory=False,
    plot_final_filtered_neuron=False,
    plot_new_axon_limb_correspondence=False,
    plot_new_limb_object=False,
    plot_final_revised_axon_branch=False,
    verbose=True,
    verbose_outline=True,
    return_limb_branch_dict_to_cancel=True,
    filter_list=pru.v4_exc_filters())

# Inspecting the Stitching

In [5]:
minnie.AutoProofreadStats5()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version key by which to lookup the decomposition process version,mesh_faces faces indices that were saved off as belonging to proofread neuron (external storage),axon_faces faces indices that were saved off as belonging to proofread neuron's axon (external storage),axon_skeleton the skeleton of the axon of the final proofread neuorn,dendrite_skeleton the skeleton of the dendrite branches of the final proofread neuorn,neuron_skeleton the skeleton of the entire neuron,axon_on_dendrite_merges_error_area the area (in um ^ 2) of the faces canceled out by filter,axon_on_dendrite_merges_error_length the length (in um) of skeleton distance canceled out by filter,low_branch_clusters_error_area the area (in um ^ 2) of the faces canceled out by filter,low_branch_clusters_error_length the length (in um) of skeleton distance canceled out by filter,dendrite_on_axon_merges_error_area the area (in um ^ 2) of the faces canceled out by filter,dendrite_on_axon_merges_error_length the length (in um) of skeleton distance canceled out by filter,double_back_and_width_change_error_area the area (in um ^ 2) of the faces canceled out by filter,double_back_and_width_change_error_length the length (in um) of skeleton distance canceled out by filter,crossovers_error_area the area (in um ^ 2) of the faces canceled out by filter,crossovers_error_length the length (in um) of skeleton distance canceled out by filter,high_degree_coordinates_error_area the area (in um ^ 2) of the faces canceled out by filter,high_degree_coordinates_error_length the length (in um) of skeleton distance canceled out by filter,high_degree_branching_error_area the area (in um ^ 2) of the faces canceled out by filter,high_degree_branching_error_length the length (in um) of skeleton distance canceled out by filter,axon_webbing_t_merges_error_area the area (in um ^ 2) of the faces canceled out by filter,axon_webbing_t_merges_error_length the length (in um) of skeleton distance canceled out by filter,thick_t_merge_error_area the area (in um ^ 2) of the faces canceled out by filter,thick_t_merge_error_length the length (in um) of skeleton distance canceled out by filter,axon_fork_divergence_error_area the area (in um ^ 2) of the faces canceled out by filter,axon_fork_divergence_error_length the length (in um) of skeleton distance canceled out by filter,n_valid_syn_presyn_for_split,n_valid_syn_postsyn_for_split,n_presyn_error_syn_non_axon,n_presyn_error_syn,n_postsyn_error_syn,total_error_synapses,total_presyns,total_postsyns,total_synapses,perc_error_presyn,perc_error_postsyn,overall_percent_error,limb_branch_to_cancel stores the limb information from,red_blue_suggestions
864691135117980637,0,0.25,30.00,5,0,0,5,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,1.46,1.33,nan,nan,nan,nan,131.12,255.36,nan,nan,nan,nan,92.85,150.4,0.0,0.0,0.0,0.0,0.0,0.0,264,359,17,23,0,23,287,359,646,0.0801,0.0,0.0356,=BLOB=,=BLOB=
864691135382556378,0,0.25,30.00,5,0,0,5,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,604.91,632.37,nan,nan,nan,nan,1037.49,985.54,nan,nan,nan,nan,2455.09,2000.1,0.0,0.0,0.0,0.0,100.1,141.54,108,678,92,125,6,131,233,684,917,0.5365,0.0088,0.1429,=BLOB=,=BLOB=
864691135517422218,0,0.25,30.00,5,0,0,5,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,365.29,269.45,nan,nan,nan,nan,2024.27,1015.92,nan,nan,nan,nan,53.91,77.32,130.89,194.62,0.0,0.0,0.0,0.0,678,158,32,36,45,81,714,203,917,0.0504,0.2217,0.0883,=BLOB=,=BLOB=
864691137019596142,0,0.25,88.00,5,0,0,5,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,9.98,16.64,nan,nan,nan,nan,0.0,0.0,nan,nan,nan,nan,182.8

In [6]:
segment_id = 864691135517422218

In [7]:
original_mesh = du.fetch_segment_id_mesh(segment_id)

In [8]:
neuron_skeleton = (minnie.AutoProofreadStats5() & dict(segment_id=segment_id)).fetch1("neuron_skeleton")
neuron_skeleton

array([[[567576.25678949, 897596.0775726 , 980727.06364785],
        [567591.41720218, 897632.30064262, 980642.86495301]],

       [[567576.25678949, 897596.0775726 , 980727.06364785],
        [567647.17474492, 897543.04359158, 980773.51836187]],

       [[567591.41720218, 897632.30064262, 980642.86495301],
        [567619.5       , 897666.        , 980553.        ]],

       ...,

       [[732327.90568207, 543145.27112836, 844650.96198287],
        [739436.        , 546626.        , 842152.        ]],

       [[732327.90568207, 543145.27112836, 844650.96198287],
        [730941.        , 550142.        , 847704.        ]],

       [[732327.90568207, 543145.27112836, 844650.96198287],
        [735669.58230103, 550673.76993182, 843560.84521343]]])

In [11]:
import neuron_visualizations as nviz
nviz.plot_objects(original_mesh,
                     skeletons=[neuron_skeleton])

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Debugging the Neuron

In [12]:
import proofreading_utils as pru

In [ ]:
# 1) Pull Down All of the Neurons
segment_id = key["segment_id"]

whole_pass_time = time.time()


curr_output = pru.proofreading_table_processing(key,
                          proof_version=proof_version,
                          axon_version  = axon_version,
                          compute_synapse_to_soma_skeletal_distance=True,
                          perform_axon_classification = False,
                          high_fidelity_axon_on_excitatory = False,
                         verbose=True,)    
# ------ Writing the Data To the Tables ----- #


AutoProofreadSynapse_keys = curr_output["AutoProofreadSynapse_keys"]
AutoProofreadSynapseErrors_keys = curr_output["AutoProofreadSynapseErrors_keys"]
AutoProofreadNeurons_keys = curr_output["AutoProofreadNeurons_keys"]
filtering_info_list = curr_output["filtering_info_list"]
synapse_stats_list = curr_output["synapse_stats_list"]
total_error_synapse_ids_list = curr_output["total_error_synapse_ids_list"]
neuron_mesh_list = curr_output["neuron_mesh_list"]
axon_mesh_list = curr_output["axon_mesh_list"]
neuron_split_idxs = curr_output["neuron_split_idxs"]

axon_skeleton_list = curr_output["axon_skeleton_list"]
dendrite_skeleton_list = curr_output["dendrite_skeleton_list"]
neuron_skeleton_list = curr_output["neuron_skeleton_list"]


# Once have inserted all the new neurons need to compute the stats
if verbose:
    print("Computing the overall stats")

overall_syn_error_rates = pru.calculate_error_rate(total_error_synapse_ids_list,
                synapse_stats_list,
                verbose=True)


# Final Part: Create the stats table entries and insert

proofread_stats_entries = []

stats_to_make_sure_in_proofread_stats = [

 'axon_on_dendrite_merges_error_area',
 'axon_on_dendrite_merges_error_length',
 'low_branch_clusters_error_area',
 'low_branch_clusters_error_length',
 'dendrite_on_axon_merges_error_area',
 'dendrite_on_axon_merges_error_length',
 'double_back_and_width_change_error_area',
 'double_back_and_width_change_error_length',
 'crossovers_error_area',
 'crossovers_error_length',
 'high_degree_coordinates_error_area',
 'high_degree_coordinates_error_length',
]


for sp_idx,split_index in enumerate(neuron_split_idxs):

    #write the AutoProofreadNeurons and AutoProofreadSynapse Tabel
    keys_to_write = AutoProofreadSynapse_keys[sp_idx]
    AutoProofreadSynapse4.insert(keys_to_write,skip_duplicates=True)

    keys_to_write_errors = AutoProofreadSynapseErrors_keys[sp_idx]
    AutoProofreadSynapseErrors4.insert(keys_to_write_errors,skip_duplicates=True)



    new_key = AutoProofreadNeurons_keys[sp_idx]
    self.insert1(new_key,skip_duplicates=True,allow_direct_insert=True)

    synapse_stats = synapse_stats_list[sp_idx]
    filtering_info = filtering_info_list[sp_idx]
    limb_branch_to_cancel = pru.extract_from_filter_info(filtering_info,
                    name_to_extract="limb_branch_dict_to_cancel")


    red_blue_suggestions = pru.extract_from_filter_info(filtering_info,
                    name_to_extract = "red_blue_suggestions")

    curr_key = dict(key,
                   split_index = split_index,
                   proof_version = proof_version,

                     mesh_faces = neuron_mesh_list[sp_idx],
                    axon_faces = axon_mesh_list[sp_idx],

                    axon_skeleton = axon_skeleton_list[sp_idx],
                    dendrite_skeleton = dendrite_skeleton_list[sp_idx],
                    neuron_skeleton = neuron_skeleton_list[sp_idx],


                    # ------------ For local valid synapses to that split_index
                    n_valid_syn_presyn_for_split=synapse_stats["n_valid_syn_presyn"],
                    n_valid_syn_postsyn_for_split=synapse_stats["n_valid_syn_postsyn"],
                    n_presyn_error_syn_non_axon=synapse_stats["n_errored_syn_presyn_non_axon"],

                    limb_branch_to_cancel = limb_branch_to_cancel,
                    red_blue_suggestions = red_blue_suggestions,


                   )


    for s in stats_to_make_sure_in_proofread_stats:
        if s not in filtering_info.keys():
            curr_key[s] = None

    filter_key = {k:np.round(v,2) for k,v in filtering_info.items() if "area" in k or "length" in k}
    curr_key.update(filter_key)
    curr_key.update(overall_syn_error_rates)

    proofread_stats_entries.append(curr_key)

# Getting the Synapses Filtered Away

In [ ]:
return_error_table_entries

keys_to_write_without_version_errors

keys_to_write_errors

AutoProofreadSynapseErrors_keys